In [51]:
import cv2 
from math import *
import math
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from numpy import loadtxt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import csv
import numpy

def toBlackWhiteBinary(img):
    for i in range(16):
        for j in range(16):
            if img[i][j] != 255:
                img[i][j] = 0
    return img

def makeBinaryImages():                
    for i in range(10):
        path = list(r"C:\Users\billy_000\Documents\UF\Neural Networks\Assign1\Neural-Networks-Assignment-1\Assignment1\images\x.PNG")
        path[104] = str(i)
        path = "".join(path)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = toBlackWhiteBinary(img)
        cv2.imwrite(path, img)
        
def ImageToArray(img):
    final = []
    for i in range(16):
        for j in range(16):
            final.append(img[i][j])
            
    for i in range(len(final)):
        if final[i] == 255:
            final[i] = 1
        else:
            final[i] = 0
    return final

def compareImage(Original, Output):
    mismatch = 0
    for i in range(len(Original)):
        if Output[i] != Original[i]:
            mismatch += 1
    return mismatch / len(Original)

def DataSetOfAllImages():
    dataset = []
    for i in range(10):
        path = list(r"C:\Users\billy_000\Documents\UF\Neural Networks\Assign1\Neural-Networks-Assignment-1\Assignment1\images\x.PNG")
        path[104] = str(i)
        path = "".join(path)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        dataset.append(ImageToArray(img))
    return dataset

def Threshold(Arr):
    newArr = []
    for i in Arr:
        if i > 0.5:
            newArr.append(1)
        else:
            newArr.append(0)
    return newArr
        
X = numpy.array(DataSetOfAllImages())
y = X
    
def Model():
    
    model = Sequential()
    model.add(Dense(256, input_dim = 256, activation='relu'))
    #model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    model.fit(X, y, epochs=150, batch_size=2, verbose = True)

    
    print(X[1])
    newY = model.predict(X)
    print(Threshold(newY[1]))
    print(compareImage(X[1], Threshold(newY[1])))
    

Model()








Epoch 1/150
5/5 [==============================] - 0s 2ms/step - loss: 4.8587
Epoch 2/150
5/5 [==============================] - 0s 3ms/step - loss: 3.6866
Epoch 3/150
5/5 [==============================] - 0s 5ms/step - loss: 3.3321
Epoch 4/150
5/5 [==============================] - 0s 7ms/step - loss: 3.2109
Epoch 5/150
5/5 [==============================] - 0s 4ms/step - loss: 3.1761
Epoch 6/150
5/5 [==============================] - 0s 4ms/step - loss: 3.1608
Epoch 7/150
5/5 [==============================] - 0s 3ms/step - loss: 3.1534
Epoch 8/150
5/5 [==============================] - 0s 6ms/step - loss: 3.1463
Epoch 9/150
5/5 [==============================] - 0s 5ms/step - loss: 3.1421
Epoch 10/150
5/5 [==============================] - 0s 3ms/step - loss: 3.1378
Epoch 11/150
5/5 [==============================] - 0s 3ms/step - loss: 3.1331
Epoch 12/150
5/5 [==============================] - 0s 4ms/step - loss: 3.1299
Epoch 13/150
5/5 [==============================] - 0s 4ms/st

5/5 [==============================] - 0s 5ms/step - loss: 3.0713
Epoch 104/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0712
Epoch 105/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0712
Epoch 106/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0712
Epoch 107/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0712
Epoch 108/150
5/5 [==============================] - 0s 5ms/step - loss: 3.0712
Epoch 109/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0711
Epoch 110/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0712
Epoch 111/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0711
Epoch 112/150
5/5 [==============================] - 0s 4ms/step - loss: 3.0711
Epoch 113/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0711
Epoch 114/150
5/5 [==============================] - 0s 3ms/step - loss: 3.0711
Epoch 115/150
5/5 [==============================] - 0